In [34]:
import numpy as np
import pickle
import json
from collections import defaultdict
import re

In [2]:
with open("reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [3]:
type(rr)

dict

In [4]:
rr.keys()

dict_keys(['312', '581', '735', '444', '670', '591', '791', '543', '749', '333', '546', '322', '459', '423', '741', '612', '677', '776', '684', '609', '537', '744', '457', '360', '531', '582', '561', '413', '605', '367', '527', '603', '595', '725', '608', '568', '648', '697', '777', '746', '316', '341', '515', '773', '562', '486', '483', '353', '307', '644', '623', '392', '710', '719', '349', '742', '398', '728', '484', '730', '469', '709', '404', '357', '419', '481', '707', '335', '763', '542', '762', '614', '580', '480', '560', '621', '737', '458', '521', '504', '687', '633', '717', '371', '449', '394', '332', '571', '662', '331', '532', '485', '525', '618', '507', '712', '442', '370', '512', '747', '718', '418', '778', '720', '647', '373', '607', '651', '421', '387', '324', '666', '461', '665', '369', '397', '407', '657', '374', '761', '545', '674', '768', '467', '694', '490', '599', '625', '688', '559', '691', '328', '402', '645', '743', '538', '344', '339', '533', '622', '733', '3

In [13]:
rr['312'].keys(), rr['612']['accepted']

(dict_keys(['id', 'conference', 'abstract', 'authors', 'histories', 'reviews', 'title', 'accepted']),
 False)

In [9]:
rr['312']['reviews'][1]

{'DATE': '19 May 2017 (modified: 02 Jun 2017)',
 'IS_ANNOTATED': True,
 'IS_META_REVIEW': False,
 'ORIGINALITY': 4,
 'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer2',
 'RECOMMENDATION_UNOFFICIAL': 5,
 'SOUNDNESS_CORRECTNESS': 5,
 'TITLE': 'Hyperparameters for SotA PTB word level LM',
 'comments': 'The paper reports that "[a]fter the controller RNN is done training, we take the best RNN cell according to the lowest validation perplexity and then run a grid search over learning rate, weight initialization, dropout rates and decay epoch. The best cell found was then run with three different configurations and sizes to increase its capacity."\n\nIs it possible for you to include (or provide here) the hyperparameters and type of dropout (i.e. recurrent dropout, embedding dropout, ...) used? Without them, replication would at best require a great deal of trial and error. As with "Recurrent Neural Network Regularization" (Zaremba et al., 2014), releasing a base set of hyper parameters great

In [11]:
uniq = []

for k in rr:
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                uniq = uniq + list(rev.keys())
                uniq = list(set(uniq))
print(uniq)

['MEANINGFUL_COMPARISON', 'DATE', 'comments', 'IMPACT', 'ORIGINALITY', 'RECOMMENDATION', 'IS_META_REVIEW', 'SUBSTANCE', 'SOUNDNESS_CORRECTNESS', 'RECOMMENDATION_UNOFFICIAL', 'IS_ANNOTATED', 'REVIEWER_CONFIDENCE', 'OTHER_KEYS', 'APPROPRIATENESS', 'TITLE', 'CLARITY']


In [35]:
acl_aspects = ["MEANINGFUL_COMPARISON", "IMPACT", "ORIGINALITY", "RECOMMENDATION", "SUBSTANCE", "SOUNDNESS_CORRECTNESS", "APPROPRIATENESS", "CLARITY"]

aspect_dict = defaultdict(dict)

for k in rr:
    new_key = rr[k]["title"].strip().lower()
    
    new_key = re.sub('[^0-9a-zA-Z]+', '', new_key)
    
    aspect_dict[new_key]["title"] = rr[k]["title"]
    aspect_dict[new_key]["dec"] = rr[k]["accepted"]
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                if rev["OTHER_KEYS"].find("AnonReviewer") > -1:
                    for asp in acl_aspects:
                        if asp in rev:
                            if asp in aspect_dict[new_key]:
                                aspect_dict[new_key][asp].append(rev[asp])
                            else:
                                aspect_dict[new_key][asp] = [rev[asp]]

In [36]:
aspect_dict

defaultdict(dict,
            {'abaselinefordetectingmisclassifiedandoutofdistributionexamplesinneuralnetworks': {'IMPACT': [2,
               2],
              'MEANINGFUL_COMPARISON': [4, 4, 4, 4],
              'ORIGINALITY': [4, 3, 4, 4, 3, 4],
              'RECOMMENDATION': [6, 6, 6, 6, 6, 6],
              'SOUNDNESS_CORRECTNESS': [3, 3],
              'dec': True,
              'title': 'A Baseline for Detecting Misclassified and Out-of-Distribution Examples in Neural Networks'},
             'acompareaggregatemodelformatchingtextsequences': {'CLARITY': [5,
               5,
               5,
               5],
              'IMPACT': [3, 3, 3, 3],
              'ORIGINALITY': [3, 3, 3, 3],
              'RECOMMENDATION': [6, 7, 8, 6, 7, 8],
              'SOUNDNESS_CORRECTNESS': [5, 5],
              'SUBSTANCE': [4, 4],
              'dec': True,
              'title': 'A Compare-Aggregate Model for Matching Text Sequences'},
             'acompositionalobjectbasedapproachtol

In [21]:
len(aspect_dict)

427

In [37]:
len(aspect_dict)

427

In [46]:
count = 0
for k in aspect_dict:
    if "APPROPRIATENESS" in aspect_dict[k]:
        count +=1
print(count)

13


In [23]:
data_dir = "../ICLR data/masterdata_unbalanced/"
year = 2017
paper_file = "papers_{}.pkl".format(year)


with open(data_dir+paper_file, "rb") as f:
    papers_data = pickle.load(f)

In [24]:
len(papers_data)

490

In [28]:
papers_data["B1-Hhnslg"]["content"]["title"]

'Prototypical Networks for Few-shot Learning'

In [39]:
final_dict = {}

not_found = []

for k in papers_data:
    t = papers_data[k]["content"]["title"].strip().lower()
    t = re.sub('[^0-9a-zA-Z]+', '', t)
    if t in aspect_dict:
        final_dict[k] = aspect_dict[t]
    else:
        print(t)
        not_found.append(t)

onlinebayesiantransferlearningforsequentialdatamodeling
neuralphotoeditingwithintrospectiveadversarialnetworks
bitpragmaticdeepneuralnetworkcomputing
geometryofpolysemy
shortanddeepsketchingandneuralnetworks
thepredictronendtoendlearningandplanning
thepowerofsparsityinconvolutionalneuralnetworks
filtershapingforconvolutionalneuralnetworks
normalizingthenormalizerscomparingandextendingnetworknormalizationschemes
alternatingdirectionmethodofmultipliersforsparseconvolutionalneuralnetworks
enforcingconstraintsonoutputswithunconstrainedinference
learningtoplayinadayfasterdeepreinforcementlearningbyoptimalitytightening
anactorcriticalgorithmforsequenceprediction
representinginferentialuncertaintyindeepneuralnetworksthroughsampling
alearnedrepresentationforartisticstyle
reasonetlearningtostopreadinginmachinecomprehension
compactembeddingofbinarycodedinputsandoutputsusingbloomfilters
deepsymbolicrepresentationlearningforheterogeneoustimeseriesclassification
warpedconvolutionsefficientinvarianc

In [40]:
len(not_found)

63

In [41]:
final_dict

{'B1-Hhnslg': {'RECOMMENDATION': [5, 6, 4, 5, 6, 4],
  'dec': False,
  'title': 'Prototypical Networks for Few-shot Learning'},
 'B1-q5Pqxl': {'CLARITY': [5, 5, 5, 5, 5, 5, 5, 5],
  'IMPACT': [3, 3, 3, 3],
  'ORIGINALITY': [5, 5, 5, 5],
  'RECOMMENDATION': [6, 7, 6, 6, 7, 6],
  'SUBSTANCE': [5, 5, 5, 5],
  'dec': True,
  'title': 'Machine Comprehension Using Match-LSTM and Answer Pointer'},
 'B16Jem9xe': {'RECOMMENDATION': [6, 8, 7, 6, 8, 7],
  'dec': False,
  'title': 'Learning in Implicit Generative Models'},
 'B16dGcqlx': {'CLARITY': [5, 5, 5, 5],
  'ORIGINALITY': [1, 1, 1, 1],
  'RECOMMENDATION': [5, 6, 6, 5, 6, 6],
  'SOUNDNESS_CORRECTNESS': [4, 4, 4, 4],
  'dec': True,
  'title': 'Third Person Imitation Learning'},
 'B184E5qee': {'CLARITY': [5, 5, 5, 5],
  'IMPACT': [2, 2, 2, 2],
  'ORIGINALITY': [2, 2, 2, 2],
  'RECOMMENDATION': [7, 5, 9, 7, 5, 9],
  'SOUNDNESS_CORRECTNESS': [4, 4, 4, 4],
  'dec': True,
  'title': 'Improving Neural Language Models with a Continuous Cache'},
 'B1

In [44]:
with open("./features/rev_aspects_2017_peerread.pkl", "wb") as f:
    pickle.dump(final_dict, f)

In [45]:
ls features/

all_data_features_17_20.pkl          data_features_csv-2017.pkl
all_data_features_csv_17_20_new.pkl  features_csv_17_20.pkl
all_data_features_csv_2017_new.pkl   rev_aspects_2017_peerread.pkl
data_csv-2017-20.pkl
